# Exemplo introdutório para combinar modelos

Este primeiro caderno visa enfatizar o benefício dos métodos de conjunto sobre
modelos simples (por exemplo, árvore de decisão, modelo linear, etc.). Combinando simples
modelos resultam em modelos mais poderosos e robustos com menos complicações.

Começaremos carregando o conjunto de dados de habitação da Califórnia. Lembramos que o
objetivo neste conjunto de dados é prever o valor médio da casa em algum distrito
na Califórnia, com base em dados demográficos e geográficos.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last"> Se você quiser uma visão geral mais detalhada sobre este conjunto de dados, pode consultar o
Apêndice - seção de descrição dos conjuntos de dados no final deste MOOC. </p>
</div>

In [2]:
from sklearn.datasets import fetch_california_housing

data, target = fetch_california_housing(as_frame=True, return_X_y=True)
target *= 100  # rescale the target in k$
target

0        452.6
1        358.5
2        352.1
3        341.3
4        342.2
         ...  
20635     78.1
20636     77.1
20637     92.3
20638     84.7
20639     89.4
Name: MedHouseVal, Length: 20640, dtype: float64

Verificaremos o desempenho estatístico do regressor de árvore de decisão com
parâmetros padrão.

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(random_state=0)
cv_results = cross_validate(tree, data, target, n_jobs=-1)
scores = cv_results["test_score"]

print(f"R2 score obtained by cross-validation: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

R2 score obtained by cross-validation: 0.354 +/- 0.087


Obtemos resultados justos. No entanto, como apresentamos anteriormente na "tree in depth", este modelo precisa ser ajustado para superar ou
insuficiente. Na verdade, os parâmetros padrão não levarão necessariamente a um
árvore de decisão ótima. Em vez de usar o valor padrão, devemos pesquisar
por meio de validação cruzada, o valor ideal dos parâmetros importantes, como
`max_depth`,` min_samples_split` ou `min_samples_leaf`.

Lembramos que precisamos ajustar esses parâmetros, pois as árvores de decisão tendem a
ajustar demais os dados de treinamento se cultivarmos árvores profundas, mas não há regras sobre
como cada parâmetro deve ser definido. Assim, não fazer uma pesquisa pode nos levar
ter um modelo insuficiente ou superaquecido.

Agora, fazemos uma pesquisa em grade para ajustar os hiperparâmetros que mencionamos
mais cedo.

In [4]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

param_grid = {
    "max_depth": [5, 8, None],
    "min_samples_split": [2, 10, 30, 50],
    "min_samples_leaf": [0.01, 0.05, 0.1, 1]}
cv = 3

tree = GridSearchCV(DecisionTreeRegressor(random_state=0),
                    param_grid=param_grid, cv=cv, n_jobs=-1)
cv_results = cross_validate(tree, data, target, n_jobs=-1,
                            return_estimator=True)
scores = cv_results["test_score"]

print(f"R2 score obtained by cross-validation: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

R2 score obtained by cross-validation: 0.523 +/- 0.107
Wall time: 8.42 s


Vemos que otimizar os hiperparâmetros terá um efeito positivo
sobre o desempenho estatístico. No entanto, ele vem com um alto desempenho computacional
custo.

Podemos criar um dataframe armazenando as informações importantes coletadas durante
o ajuste dos parâmetros e investigação dos resultados.

Agora usaremos um método de ensemble chamado bagging. Mais detalhes sobre isso
método será discutido na próxima seção. Em suma, este método usará
um regressor de base (ou seja, regressores de árvore de decisão) e treinará vários dos
em uma versão ligeiramente modificada do conjunto de treinamento. Então o
as previsões de todos esses regressores de base serão combinadas pela média.

Aqui, usaremos 50 árvores de decisão e verificaremos o tempo de ajuste, bem como o
desempenho estatístico nos dados de teste deixados de fora. É importante notar
que não vamos ajustar nenhum parâmetro da árvore de decisão.

In [5]:
%%time
from sklearn.ensemble import BaggingRegressor

base_estimator = DecisionTreeRegressor(random_state=0)
bagging_regressor = BaggingRegressor(
    base_estimator=base_estimator, n_estimators=20, random_state=0)

cv_results = cross_validate(bagging_regressor, data, target, n_jobs=-1)
scores = cv_results["test_score"]

print(f"R2 score obtained by cross-validation: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

R2 score obtained by cross-validation: 0.642 +/- 0.083
Wall time: 3.05 s


Sem procurar hiperparâmetros ideais, a estatística geral
o desempenho do regressor de bagging é melhor do que uma única árvore de decisão.
Além disso, o custo computacional é reduzido em comparação com a busca
para os hiperparâmetros ideais.

Isso mostra a motivação por trás do uso de um aluno de conjunto: dá uma
linha de base relativamente boa com desempenho estatístico decente, sem qualquer
ajuste de parâmetro.

Agora, vamos discutir em detalhes dois ensemble families: bagging and
boosting:

* ensemble using bootstrap (e.g. bagging and random-forest);
* ensemble using boosting (e.g. adaptive boosting and gradient-boosting
  decision tree).